In [10]:
import pandas as pd 

Pour cette exploration, nous allons nous concentrer sur un échantillon de 2020. 
Si besoin, nous pourrons répéter le même processus pour d'autres années, mais les changements étant mineurs, les résultats devraient être le même. 

Cet échantillon contient 998272 lignes budgets et représente 7335 fichiers.

On va résumer ça à 1 million de lignes et 7k fichiers représentés. 

En isolant les lignes possédant une fonction, nous obtenons environ 550k lignes.

- Parmis ces lignes, 496000 cas où Fonction_compte ne suffit pas 
------------------

# Resume de l'exploration 

----------
### Fonctions : 

- Lors d'une jointure sur ces 550k lignes avec fonction_compte, 496000 n'ont pas de correspondance. Lorsqu'on fait un nouveau join avec chap, rien ne change, si on le fait avec fonction_ref, on passe à 30k lignes sans correspondance. 

- Environ 18% des fonctions sont donc mal encodées sur notre échantillon. 

- Si on fusionne directement les dataframe fonction_compte et fonction_ref aux lignes budgets, alors certains codes se chevaucheront : compte et ref peuvent avoir les mêmes codes mais avec des libelles différents. Pour combler ça il faut donc faire l'un puis l'autre uniquement sur les lignes manquantes, question : à qui donner la priorité ? 

- En essayant de reproduire la méthode qu'avait trouvé Damien (sans avoir son code), plusieurs cas suivent la logique suivante :
    - Si la fonction dans les lignes_budget contient 4 chiffres (1234), retirer le dernier (123) puis chercher y ajouter 9X. On tombe sur des choses cohérentes mais quasi constamment redondantes. Ex : en cherchant 9X123 (pour la fonction orpheline 1234) on tombe sur 90123 et 93123 / 92123, les trois ont le même libelle, mais 90123 aura Dequip et Requip en TRUE et les deux autres en false.
    - Si la fonction dans les lignes_budget contient 3 chiffres (123), faire la même chose sans retirer.     
- Cependant. Certains cas ne correspondent pas à ce que nous attendons : 123 peut aussi se référer à 9123 ET 90123, voire très certainement d'autres qui ne sont pas en 9X (continuer l'exploration pour ça.)

- Même si on pouvait arbitrer sur le choix du 9X à choisir, on ne peut pas simplement supprimer 9XXXX des fonction_compte, car plusieurs lignes ont des fonctions correctement encodées, correspondant directement à 9XXXX

- Etant donné que je ne peux pas certifier que la méthode "9X" fonctionne dans tout les cas (en fait, il y a même plusieurs cas où elle ne fonctionne pas), je recommande de ne pas y toucher, l'absence d'une information secondaire est moins problématique que la création d'une fausse information. 

Voici le resultat des données manquantes : 

In [125]:
pd.read_csv('./comptage_fonction_sans_correspondance.csv', index_col= [0])

,Nomenclature,nb de lignes,Nb de fonctions orphelines,Nb de lignes orphelines,% fonctions orphelines
0,M14-M14_COM_SUP3500,467788,211,2776,0.59
1,M52-M52,32473,6,11,0.03
2,M57-M57,28319,7,181,0.64
3,M14-M14_CCAS_SUP3500,14847,10,88,0.59
4,M71-M71,1185,2,2,0.17
5,M14-M14_CCAS_INF3500,221,13,221,100.00
6,M14-M14_CE,22,1,22,100.00


### Nature 

- Toutes les lignes budgets ont une nature, c'est évident mais on sait jamais. 
- Il y a 1749 lignes budgets où la nature n'a pas de correspondances, soit 0.175% de l'échantillon. 
- Pas encore trouvé de solution pour celles qui sont orphelines. 


In [143]:
pd.read_csv('./comptage_nature_orphelines.csv', index_col=[0])

,Nomenclature,Nb de lignes,Nb de lignes orphelines,Nb de Natures orphelines,% lignes affectées
0,M14-M14_COM_SUP3500,487455,292,202,0.060
1,M14-M14_COM_500_3500,311813,188,82,0.060
2,M4-M49_D,38680,501,496,1.295
3,M14-M14_COM_INF500,35067,572,118,1.631
4,M52-M52,33195,3,3,0.009
5,M57-M57,28608,25,19,0.087
6,M4-M49_A,16897,109,42,0.645
7,M4-M4,15808,32,25,0.202
8,M14-M14_CCAS_SUP3500,15024,16,10,0.106
9,M14-M14_CCAS_INF3500,6004,7,6,0.117


In [12]:
df_2020 = pd.read_parquet('./parquet/echantillon_2020.parquet')

In [27]:
print('Nombre de lignes :',df_2020.shape[0])
print('Nombre de fichiers : ', df_2020['Id_Fichier'].drop_duplicates().shape[0])

Nombre de lignes : 998272
Nombre de fichiers :  7335


On va donc travailler sur un échantillon de 7335 fichiers, ce qui nous donne presque un million de lignes. 

In [15]:
#n pour nature et f pour compte. 

nchap = pd.read_parquet('./parquet/Nature_Chapitre_2020.parquet')
ncompte = pd.read_parquet('./parquet/Nature_Compte_2020.parquet')
fchap = pd.read_parquet('./parquet/Fonction_Chapitre_2020.parquet')
fcompte = pd.read_parquet('./parquet/Fonction_Compte_2020.parquet')
fref = pd.read_parquet('./parquet/Fonction_Referentiel_2020.parquet')

Rappel de la structure des df contenant les libelles : 

In [16]:
nchap.head(1)

,Code,Lib_court,Libelle,PourEtatSeul,Section,Special,TypeChapitre,Exer,Nomenclature
0,002,Résultat de fonctionnement reporté,Résultat de fonctionnement reporté,None,F,0,L,2020,M14-M14_COM_500_3500


In [17]:
ncompte.head(1)

,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,Exer,Nomenclature,PourEtatSeul
0,001,0,,,001,Solde d'exécution sect° d'investissement,Solde d'exécution de la section d'investisseme...,0,,,001,None,None,None,2020,M14-M14_COM_500_3500,None


On retire les données de Montants pour n'avoir que les données nécessaire lors de notre exploration


In [18]:
df_budget = df_2020[['Id_Fichier', 'Nomenclature', 'TypOpBudg', 'Operation', 'Nature', 'ContNat',
              'LibCpte', 'Fonction', 'ContFon', 'ArtSpe']]

df_fonction = df_budget[~df_budget['Fonction'].isna()] #On conserve que les lignes avec une fonction
df_budget.head(1)

,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,ContFon,ArtSpe
0,656416,M14-M14_COM_500_3500,None,None,60612,011,Energie - Electricité,None,None,False


In [43]:
df_f_merge = df_fonction.merge(fcompte, 
                left_on = ['Nomenclature', 'Fonction'], 
                right_on = ['Nomenclature', 'Code'], 
                how = 'left')

df_f_merge = df_f_merge[['Id_Fichier', 'Nomenclature', 'TypOpBudg', 
                         'Operation', 'Nature', 'ContNat', 
                         'Fonction', 'Lib_court']]

#df_fsc = lignes budgets contenant une fonction qui n'a pas de correspondance dans fonction compte
df_fsc = df_f_merge[df_f_merge['Lib_court'].isna()]
print('Lignes avec fonction :', df_f_merge.shape[0])
print('Lignes avec fonction sans correspondance :',df_fsc.shape[0])
print("Pourcentage d'absence :", int((len(df_fsc) / len(df_f_merge) * 100)), '%')

Lignes avec fonction : 544855
Lignes avec fonction sans correspondance : 496000
Pourcentage d'absence : 91 %


In [20]:
liste_sans_fcompte = df_f_merge[df_f_merge['Lib_court'].isna()]['Fonction'].drop_duplicates().sort_values().to_list()

Exploration d'un des cas qui n'a pas de correspondance avec fonction_compte

In [46]:
df_f_merge[df_f_merge['Fonction'] == '001']

,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,Fonction,Lib_court
26740,777683,M71-M71,None,None,001,None,001,NaN


In [21]:
#Ce n'est pas non plus dans les trois autres, ce code n'existe pas en 2020 pour les fonctions
fcompte[fcompte['Code'] == '001']

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis


001, qui est une fonction orpheline a une correspondance avec Nature, ça ne veut pourtant pas dire pour autant qu'on peut la remplacer par la Nature 001. On ne peut que attester du fait que c'est une erreur d'encodage quelque part, garbage in. 

In [22]:
nchap[(nchap['Code'] == '001') & (nchap['Nomenclature'] == 'M71-M71')]

,Code,Lib_court,Libelle,PourEtatSeul,Section,Special,TypeChapitre,Exer,Nomenclature
22,001,Solde exécution section investissement,Solde d'exécution de la section d'investisseme...,None,I,0,L,2020,M71-M71


In [23]:
df_budget[(df_budget['Fonction'] == '001') & (df_budget['Nomenclature'] == 'M52-M52')]

,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,ContFon,ArtSpe


In [24]:
df_budget[(df_budget['Fonction'] == '001')]

,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,ContFon,ArtSpe
45430,777683,M71-M71,None,None,001,None,None,001,None,False


In [48]:
df_fscdf_f_cmch = df_fsc.merge(fchap, left_on = ['Nomenclature', 'Fonction'], right_on = ['Nomenclature', 'Code'], suffixes = ('_compte', '_chap'), how = 'left')

496000

In [50]:
df_f_cmref = df_f_merge.merge(fref, left_on = ['Nomenclature', 'Fonction'], right_on = ['Nomenclature', 'Code'], suffixes = ('_compte', '_ref'), how = 'left')
sous_df_ref = df_fsc.merge(fref, left_on = ['Nomenclature', 'Fonction'], right_on = ['Nomenclature', 'Code'], suffixes = ('_compte', '_ref'), how = 'left')
df_f_cmref[df_f_cmref['Lib_court_ref'].isna()].shape[0]

28798

Malheureusement, on ne peut pas faire les join avec fonction_compte et fonction_ref directement, les deux peuvent se chevaucher avec des libelle différents 

In [51]:
#LES DEUX SE CHAUVAUCHENT 
df_f_cmref[(~df_f_cmref['Lib_court_compte'].isna()) & (~df_f_cmref['Lib_court_ref'].isna())]\
  [['Id_Fichier', 'Nomenclature', 'Nature', 'ContNat', 'Fonction', 'Lib_court_compte', 'Lib_court_ref']]

,Id_Fichier,Nomenclature,Nature,ContNat,Fonction,Lib_court_compte,Lib_court_ref
863,758582,M14-M14_COM_SUP3500,70323,70,90,Opérations d'équipement,Interventions économiques
883,758582,M14-M14_COM_SUP3500,7336,73,91,Opérations non ventilées,Foires et marchés
1186,734372,M14-M14_COM_SUP3500,60632,011,91,Opérations non ventilées,Foires et marchés
1355,734372,M14-M14_COM_SUP3500,6281,011,92,Services individualisés,Aides à l'agriculture et aux industries
1394,734372,M14-M14_COM_SUP3500,7336,None,91,Opérations non ventilées,Foires et marchés
...,...,...,...,...,...,...,...
544656,639621,M14-M14_COM_SUP3500,6215,012,90,Opérations d'équipement,Interventions économiques
544658,639621,M14-M14_COM_SUP3500,6064,011,90,Opérations d'équipement,Interventions économiques
544659,639621,M14-M14_COM_SUP3500,7083,None,90,Opérations d'équipement,Interventions économiques
544660,639621,M14-M14_COM_SUP3500,7815,None,90,Opérations d'équipement,Interventions économiques


On va compter les fonctions qui ont n'ont pas de correspondance par nomenclature en 2020 

In [52]:
df_trou = df_f_cmref[(df_f_cmref['Lib_court_compte'].isna()) & (df_f_cmref['Lib_court_ref'].isna())]
df_trou1 = df_trou.drop_duplicates(subset=['Nomenclature', 'Fonction'])
vc1 = df_trou1['Nomenclature'].value_counts()
vc = df_trou['Nomenclature'].value_counts()

In [112]:
vc_lignes_fonc = df_f_cmref['Nomenclature'].value_counts()
dfvc_lignes_fonc = pd.DataFrame(vc_lignes_fonc).reset_index().rename(columns = {'count' : 'nb de lignes'})

,Nomenclature,nb de lignes
0,M14-M14_COM_SUP3500,467788
1,M52-M52,32473
2,M57-M57,28319
3,M14-M14_CCAS_SUP3500,14847
4,M71-M71,1185
5,M14-M14_CCAS_INF3500,221
6,M14-M14_CE,22


In [136]:
df_vc1 = pd.DataFrame(vc1).reset_index().rename(columns = {'count' : 'Nb de fonctions orphelines'})
df_vc = pd.DataFrame(vc).reset_index().rename(columns = {'count' : 'Nb de lignes orphelines'})

df_comptage = dfvc_lignes_fonc.merge(df_vc1, on = 'Nomenclature', how = 'left')\
                              .merge(df_vc, on = 'Nomenclature', how = 'left')
df_comptage['% fonctions orphelines'] = (df_comptage['Nb de lignes orphelines'] / df_comptage['nb de lignes'] * 100).round(2)
df_comptage.to_csv('./csv/comptage_fonction_sans_correspondance.csv')
df_comptage

,Nomenclature,nb de lignes,Nb de fonctions orphelines,Nb de lignes orphelines,% fonctions orphelines
0,M14-M14_COM_SUP3500,467788,211,2776,0.59
1,M52-M52,32473,6,11,0.03
2,M57-M57,28319,7,181,0.64
3,M14-M14_CCAS_SUP3500,14847,10,88,0.59
4,M71-M71,1185,2,2,0.17
5,M14-M14_CCAS_INF3500,221,13,221,100.00
6,M14-M14_CE,22,1,22,100.00


In [58]:
liste_trou = df_trou1['Fonction'].to_list()
print('Nombre de fonctions affectés',len(liste_trou))
print('Exemple de fonctions',liste_trou[0:5])

Nombre de fonctions affectés 250
Exemple de fonctions ['0209', '4121', '4131', '4141', '0203']


In [60]:
df_budget[df_budget['Fonction'] == '0209'].head(5)

,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,ContFon,ArtSpe
32958,690898,M14-M14_COM_SUP3500,None,None,60611,None,None,0209,None,False
34613,690898,M14-M14_COM_SUP3500,None,920,2031,None,TRAVAUX DE PROXIMITE,0209,None,False
34627,690898,M14-M14_COM_SUP3500,None,442,2033,None,CENTRE HAUSSMANN,0209,None,False
34629,690898,M14-M14_COM_SUP3500,None,920,2033,None,TRAVAUX DE PROXIMITE,0209,None,False
34659,690898,M14-M14_COM_SUP3500,None,442,21318,None,CENTRE HAUSSMANN,0209,None,False


Cherchons donc les dérivés de  0209 dans les fonctions 

In [63]:
fcompte[(fcompte['Code'].str.contains('020')) & (fcompte['Nomenclature'] == 'M14-M14_COM_SUP3500')]

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis
3,2020,M14-M14_COM_SUP3500,90020,1,,,900,Administration générale collectivité,Administration générale de la collectivité,1,,,900,None,None,None
23,2020,M14-M14_COM_SUP3500,9020,1,,,902,Services communs,Services communs,1,,,902,None,None,None
126,2020,M14-M14_COM_SUP3500,92020,0,,,920,Administration générale collectivité,Administration générale de la collectivité,0,,,920,None,None,None


209 est introuvable. 
Pour 020, nous avons 3 correspondances possibles dans les fonctions compte. Besoin d'arbitrage. 

Testons avec un autre 

In [64]:
df_budget[df_budget['Fonction'] == '3121'].head(4)

,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,ContFon,ArtSpe
34688,690898,M14-M14_COM_SUP3500,None,920,2135,None,TRAVAUX DE PROXIMITE,3121,None,False
34775,690898,M14-M14_COM_SUP3500,None,950,2183,None,INFORMATIQUE,3121,None,False
34842,690898,M14-M14_COM_SUP3500,None,None,6042,None,None,3121,None,False
34865,690898,M14-M14_COM_SUP3500,None,None,60612,None,None,3121,None,False


In [65]:
fcompte[(fcompte['Nomenclature'] == 'M14-M14_COM_SUP3500') & (fcompte['Code'].str.contains('312'))]

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis
41,2020,M14-M14_COM_SUP3500,90312,1,,,903,Arts plastiques et activités artistiques,Arts plastiques et autres activités artistiques,1,,,903,None,None,None
164,2020,M14-M14_COM_SUP3500,92312,0,,,923,"Arts plastiques, activités artistiques",Arts plastiques et autres activités artistiques,0,,,923,None,None,None


Problème : La nature 2135 correspond à "Installations générales, agencements, aménagements des constructions". 

Pas sûr que les cours d'arts plastiques aient beaucoup de rapports. 

In [ ]:
fcompte[(fcompte['Nomenclature'] == 'M14-M14_COM_SUP3500') & (fcompte['Code'].str.contains('312'))]

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis
41,2020,M14-M14_COM_SUP3500,90312,1,,,903,Arts plastiques et activités artistiques,Arts plastiques et autres activités artistiques,1,,,903,None,None,None
164,2020,M14-M14_COM_SUP3500,92312,0,,,923,"Arts plastiques, activités artistiques",Arts plastiques et autres activités artistiques,0,,,923,None,None,None


In [72]:
df_f_cmref[['Id_Fichier', 'Nomenclature', 'TypOpBudg', 'Operation', 'Nature', 'ContNat', 'Fonction', 'Lib_court_compte', 'Lib_court_ref']].head(2)

,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,Fonction,Lib_court_compte,Lib_court_ref
0,747133,M14-M14_COM_SUP3500,None,None,7015,None,9,NaN,Action économique
1,747133,M14-M14_COM_SUP3500,Opération de section à section,None,3555,040,01,NaN,Opérations non ventilables


Passage en csv d'un DataFrame contenant une jointure avec fonction_compte et une autre avec fonction_ref 

In [67]:
sous_df_ref.to_csv('./csv/fonction_correspondance_ref.csv')

Csv contenant uniquement les lignes où fonction n'a de correspondance ni avec ref, ni avec compte, ni avec chapitre, bref, les véritables fonctions orphelines 

In [73]:
sous_df_vide = sous_df_ref[(sous_df_ref['Lib_court_compte'].isna()) & (sous_df_ref['Lib_court_ref'].isna())]
sous_df_vide[['Id_Fichier', 'Nomenclature', 'TypOpBudg', 
              'Operation','Nature', 'ContNat', 'Fonction', 
              'Lib_court_compte', 'Lib_court_ref']]\
            .to_csv('./csv/fonction_sans_correspondances_2020.csv')

In [79]:
liste_pure_vide = sous_df_vide.drop_duplicates(subset = ['Nomenclature', 'Fonction'])['Fonction'].to_list()
print('Nb de fonctions orpelines',len(liste_pure_vide))
print('Exemple : ', liste_pure_vide[0:5])

Nb de fonctions orpelines 250
Exemple :  ['0209', '4121', '4131', '4141', '0203']


In [ ]:
fcompte[(fcompte['Code'].str.contains('414')) & (fcompte['Nomenclature'] == 'M14-M14_COM_SUP3500')]
#Nature 60611 correspond à eau et assainissement, donc ça devrait le faire, mais on sait pas si c'est 92 ou 90.. 
#C'est globalement pareil sur les autres 41XX. On retire le dernier chiffre et on rajoute 90 / 93 

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis
56,2020,M14-M14_COM_SUP3500,90414,1,,,904,Autres équipements sportifs ou de loisir,Autres équipements sportifs ou de loisir,1,,,904,None,None,None
179,2020,M14-M14_COM_SUP3500,92414,0,,,924,Autres équipements sportifs ou de loisir,Autres équipements sportifs ou de loisir,0,,,924,None,None,None


In [ ]:
#0203
fcompte[(fcompte['Code'].str.contains('020')) & (fcompte['Nomenclature'] == 'M14-M14_COM_SUP3500')]
#Nature 2031 correspond à des frais d'études, on tome sur la même logique 

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis
3,2020,M14-M14_COM_SUP3500,90020,1,,,900,Administration générale collectivité,Administration générale de la collectivité,1,,,900,None,None,None
23,2020,M14-M14_COM_SUP3500,9020,1,,,902,Services communs,Services communs,1,,,902,None,None,None
126,2020,M14-M14_COM_SUP3500,92020,0,,,920,Administration générale collectivité,Administration générale de la collectivité,0,,,920,None,None,None


In [ ]:
#204
fcompte[(fcompte['Code'].str.contains('204')) & (fcompte['Nomenclature'] == 'M14-M14_COM_SUP3500')]
#Nature 60623 = Alimentation. Donc les deux correspondent 

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis
134,2020,M14-M14_COM_SUP3500,9204,None,None,None,None,"Coopérat° décentralisée, act° européenne","Coopération décentralisée, actions européennes...",None,None,None,None,1,None,None
135,2020,M14-M14_COM_SUP3500,92041,0,,,920,Subvention globale,Subvention globale,0,,,920,None,None,None
136,2020,M14-M14_COM_SUP3500,92048,0,,,920,Autres act° coopération décentralisée,Autres actions de coopération décentralisée,0,,,920,None,None,None


In [81]:
#2112
fcompte[(fcompte['Nomenclature'] == 'M14-M14_COM_SUP3500') & (fcompte['Code'].str.contains('211'))].head(3)

,Exer,Nomenclature,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis
25,2020,M14-M14_COM_SUP3500,90211,1,,,902,Ecoles maternelles,Ecoles maternelles,1,,,902,None,None,None
138,2020,M14-M14_COM_SUP3500,9211,None,None,None,None,Sécurité intérieure,Sécurité intérieure,None,None,None,None,1,None,None
139,2020,M14-M14_COM_SUP3500,92110,0,,,921,Services communs,Services communs,0,,,921,None,None,None


In [80]:
#2112
sous_df_vide[sous_df_vide['Fonction'] == '2112'].head(3)


,Id_Fichier,Nomenclature,TypOpBudg,Operation,Nature,ContNat,Fonction,Lib_court_compte,Exer,Code,Lib_court_ref,Libelle
18149,690898,M14-M14_COM_SUP3500,None,920,2031,None,2112,NaN,NaN,NaN,NaN,NaN
18189,690898,M14-M14_COM_SUP3500,None,920,21312,None,2112,NaN,NaN,NaN,NaN,NaN
18218,690898,M14-M14_COM_SUP3500,None,920,2135,None,2112,NaN,NaN,NaN,NaN,NaN


Verification d'un petit détail : Est ce qu'il y a des fonctions 90XXX dans les lignes_budgets ?

In [ ]:
df_budget_M14 = df_budget[df_budget['Nomenclature'] == 'M14-M14_COM_SUP3500']

df_budget_M14_90 = df_budget_M14[(df_budget_M14['Fonction'].str.startswith('90')) & 
              (~df_budget_M14['Fonction'].isna())]
df_budget_M14_90['Fonction'].value_counts()

Fonction
90       11430
90020      114
90824       73
90822       71
90412       50
         ...  
901          1
90511        1
90253        1
90512        1
902          1
Name: count, Length: 76, dtype: int64

Donc on ne peut pas juste couper le 90 des codes de fonction_compte 

# Maintenant on s'attaque aux natures 

In [90]:
#nc = nature compte
df_budget_nc = df_budget.merge(ncompte, 
                left_on=['Nomenclature', 'Nature'], 
                right_on = ['Nomenclature', 'Code'], 
                how = 'left')

df_budget_nc = df_budget_nc[['Id_Fichier', 'Nomenclature','TypOpBudg',
                             'Operation','LibCpte','Nature',
                             'Lib_court', 'Fonction']]

df_budget_nc.head(2)

,Id_Fichier,Nomenclature,TypOpBudg,Operation,LibCpte,Nature,Lib_court,Fonction
0,656416,M14-M14_COM_500_3500,None,None,Energie - Electricité,60612,Energie - Electricité,None
1,656416,M14-M14_COM_500_3500,None,None,Combustibles,60621,Combustibles,None


In [99]:
df_nature_orpheline = df_budget_nc[df_budget_nc['Lib_court'].isna()]

print('Lignes budget sans nature :',(df_budget_nc[df_budget_nc['Nature'].isna()].shape[0]))
print('Nature sans correspondance dans Nature_Compte : ',
      (df_nature_orpheline.shape[0]))
print('Pourcentage de lignes affectées : ', 
      round((len(df_nature_orpheline) / len(df_budget_nc) * 100), 3), '%'
      )

Lignes budget sans nature : 0
Nature sans correspondance dans Nature_Compte :  1749
Pourcentage de lignes affectées :  0.175 %


Comme pour les fonctions, faisons un état des lieux des nomenclature affectées. 

In [137]:
vc_nature_budget = df_budget_nc['Nomenclature'].value_counts()
df_vc_nabu = pd.DataFrame(vc_nature_budget).reset_index().rename(columns = {'count' : 'Nb de lignes'})
df_vc_nabu

,Nomenclature,Nb de lignes
0,M14-M14_COM_SUP3500,487455
1,M14-M14_COM_500_3500,311813
2,M4-M49_D,38680
3,M14-M14_COM_INF500,35067
4,M52-M52,33195
5,M57-M57,28608
6,M4-M49_A,16897
7,M4-M4,15808
8,M14-M14_CCAS_SUP3500,15024
9,M14-M14_CCAS_INF3500,6004


In [141]:
tab_nat_orph_lignes = df_nature_orpheline['Nomenclature'].value_counts()
tab_nat_orph = df_nature_orpheline.drop_duplicates(
                ['Nomenclature', 'Nature'])['Nomenclature'].value_counts()

df_nature_orph1 = pd.DataFrame(tab_nat_orph_lignes).reset_index()\
                              .rename(columns = {'count' : 'Nb de lignes orphelines'})
df_nature_orph_unique = pd.DataFrame(tab_nat_orph).reset_index()\
                              .rename(columns = {'count' : 'Nb de Natures orphelines'})

dfn_orph = df_vc_nabu.merge(df_nature_orph1, on ='Nomenclature', how= 'left')\
          .merge(df_nature_orph_unique, on = 'Nomenclature', how= 'left').fillna(0)
dfn_orph['Nb de lignes orphelines'] = dfn_orph['Nb de lignes orphelines'].astype(int)
dfn_orph['Nb de Natures orphelines'] = dfn_orph['Nb de Natures orphelines'].astype(int)
dfn_orph['% lignes affectées'] = round((dfn_orph['Nb de lignes orphelines'] / dfn_orph['Nb de lignes']) * 100,3)
dfn_orph.to_csv('./csv/comptage_nature_orphelines.csv')

Exploration des lignes budgets où la Nature ne trouve pas de correspondance dans nature_compte 

In [95]:
df_budget_nc[df_budget_nc['Lib_court'].isna()].head(5)

,Id_Fichier,Nomenclature,TypOpBudg,Operation,LibCpte,Nature,Lib_court,Fonction
2702,623838,M14-M14_COM_INF500,None,19,Aménagement place,2031,NaN,None
2703,623838,M14-M14_COM_INF500,None,240,Mairie-SDF,2031,NaN,None
2725,623838,M14-M14_COM_INF500,None,240,Mairie-SDF,2313,NaN,None
2777,623838,M14-M14_COM_INF500,None,220,Forêt,1321,NaN,None
2778,623838,M14-M14_COM_INF500,None,240,Mairie-SDF,1323,NaN,None


Mauvaise nouvelle, la nature 2031 ne trouve pas non plus de correspondance dans Nature_chapitre (on sait jamais)

In [96]:
nchap[nchap['Code'].str.contains('2031')]

,Code,Lib_court,Libelle,PourEtatSeul,Section,Special,TypeChapitre,Exer,Nomenclature


In [97]:
ncompte[(ncompte['Code'].str.contains('2031')) & (ncompte['Nomenclature'] == 'M14-M14_COM_INF500')]

,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,Exer,Nomenclature,PourEtatSeul


Si on applique la même logique que fonction : 

In [98]:
ncompte[(ncompte['Code'].str.contains('203')) & (ncompte['Nomenclature'] == 'M14-M14_COM_INF500')]

,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,Exer,Nomenclature,PourEtatSeul
104,203,1,040,041,20,"Frais d'études, recherche, développement","Frais d'études, de recherche et de développeme...",1,040,041,20,None,None,None,2020,M14-M14_COM_INF500,None


On tombe sur de la R&D pour... l'aménagement d'une place 

In [88]:
#tentons l'inverse : chercher par place / aménagement

ncompte[ncompte['Lib_court'].str.contains('place ')]
ncompte[ncompte['Lib_court'].str.contains(' aménagement')]

,Code,DEquip,DOES,DOIS,DR,Lib_court,Libelle,REquip,ROES,ROIS,RR,RegrTotalise,Supprime,SupprimeDepuis,Exer,Nomenclature,PourEtatSeul
290,212,None,None,None,None,Agencements et aménagements de terrains,Agencements et aménagements de terrains,None,None,None,None,None,None,None,2020,M14-M14_COM_500_3500,None
292,2128,1,040,041,21,Autres agencements et aménagements,Autres agencements et aménagements,1,040,041,21,None,None,None,2020,M14-M14_COM_500_3500,None
417,2312,1,040,041,23,Agencements et aménagements de terrains,Agencements et aménagements de terrains,1,040,041,23,None,None,None,2020,M14-M14_COM_500_3500,None
569,2812,None,None,None,None,Agencements et aménagements de terrains,Agencements et aménagements de terrains,None,None,None,None,None,None,None,2020,M14-M14_COM_500_3500,None
571,28128,0,040,,,Autres aménagements de terrains,Autres agencements et aménagements de terrains,0,040,,,None,None,None,2020,M14-M14_COM_500_3500,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,28128,0,040,,,Autres aménagements de terrains,Autres agencements et aménagements de terrains,0,040,,,None,None,None,2020,M14-M14_CCAS_INF3500,None
154,212,None,None,None,None,Agencements et aménagements de terrains,Agencements et aménagements de terrains,None,None,None,None,None,None,None,2020,M14-M14_CE,None
156,2128,1,040,041,21,Autres agencements et aménagements,Autres agencements et aménagements,1,040,041,21,None,None,None,2020,M14-M14_CE,None
252,2812,None,None,None,None,Agencements et aménagements de terrains,Agencements et aménagements de terrains,None,None,None,None,None,None,None,2020,M14-M14_CE,None
